In [1]:
import numpy as np

from qiskit import Aer, execute, QuantumCircuit
from qiskit.circuit import ParameterVector

from qiskit.aqua.operators import MatrixOp, StateFn

from qiskit.aqua.operators.converters import CircuitSampler
from qiskit.aqua.operators.expectations import PauliExpectation


# define Matrix - and print eigenvalues
M = np.array([[1,  0,  0, 0], 
              [0,  0, -1, 0], 
              [0, -1,  0, 0], 
              [0,  0,  0, 1]])

np.linalg.eig(M)

(array([ 1., -1.,  1.,  1.]),
 array([[ 0.        ,  0.        ,  1.        ,  0.        ],
        [-0.70710678,  0.70710678,  0.        ,  0.        ],
        [ 0.70710678,  0.70710678,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

### Decompose M into  a sum of Pauli matrices
M is a Hermitian Matrix and naturally coresponds to a Hamiltonian operator $ H $. We decompose the operator to a sum of Pauli matrices, $ H = \sum_a w_a P_a $, since we can estimate the expectation value of a Pauli operator through measurements in computational basis and finally estimate the expectation value of H as

$$ \langle H \rangle = \sum_a w_a \langle P_a \rangle $$

In [2]:
H = MatrixOp(M).to_pauli_op()

print(H)

SummedOp(
[0.5 * II,
-0.5 * XX,
-0.5 * YY,
0.5 * ZZ])


### Ansatz
We choose the following simple variatonal form

In [3]:
# define variatonal form
def ansatz():
    
    phi = ParameterVector(name='phi', length=2)
    
    qc = QuantumCircuit(2)
    qc.ry(phi[0], 0)
    qc.ry(phi[1], 1)
    qc.cx(0, 1)    
    
    return qc, phi


qc, params = ansatz()

print('Variatonal form')
print(qc)

Variatonal form
     ┌────────────┐     
q_0: ┤ RY(phi[0]) ├──■──
     ├────────────┤┌─┴─┐
q_1: ┤ RY(phi[1]) ├┤ X ├
     └────────────┘└───┘


### Cost function
The cost is simply the expectation value of $ H $.

We measure the expectation value of the Hamiltonian $ H $ using `CircuitSampler` and `PauliExpectation` objects built in Qiskit-Aqua. In short,

- `PauliExpectation` groups the Pauli terms in $ H $ and prepares the circuits for measurements in $ Z, X, Y $ basis. 
- `CircuitSampler` executes the circuits and computes the expectation values.

In [4]:
BACKEND = Aer.get_backend('qasm_simulator')

sampler = CircuitSampler(BACKEND)
exp = PauliExpectation(group_paulis=True)

qc, params = ansatz()

def cost(phi):
    
    # circuit 
    circ = qc.bind_parameters(dict(zip(params, phi)))
    
    # execute and measure expectation value of H
    cost = sampler.convert(exp.convert(~StateFn(H)) @ StateFn(circ)).eval()
    
    return cost.real

### Optimization

In [7]:
from scipy.optimize import minimize

# random initial choice
phi = np.random.uniform(0, 2 * np.pi, len(params))

res = minimize(cost, phi, 
               method='COBYLA')

print('\nLowest eigenvalue: ', res.fun)
res


Lowest eigenvalue:  -1.0


     fun: -1.0
   maxcv: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 32
  status: 1
 success: True
       x: array([7.8271194 , 3.13380394])

## VQE in Qiskit-Aqua
For demonstration purposes only, we use the same variatonal form but the VQE instance already built in Qiskit Aqua.

In [8]:
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.aqua import QuantumInstance

optimizer = SPSA(max_trials=100)

var_form, _ = ansatz()

vqe = VQE(H, var_form, optimizer)
vqe_result = vqe.run(QuantumInstance(backend=BACKEND))

vqe_result

print('\nLowest eigenvalue: ', vqe_result.eigenvalue)


Lowest eigenvalue:  (-1+0j)
